In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
# import cv2
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-12-04 16:13:58.127635: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
raw_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_images = np.load('train_images.npy')
train_ids = np.load('train_ids.npy')

train_labels = raw_data['price'].values
train_labels = train_labels[train_ids]
y_cat = to_categorical(train_labels, num_classes = 6)  # Adjust labels for categorical crossentropy

In [4]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [5]:
train_images = train_images / 255

In [7]:
train_datagen.fit(train_images)

In [8]:
train_images.shape, y_cat.shape

((15096, 224, 224, 3), (15096, 6))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train_images, y_cat, test_size = 0.2, random_state = 42)

In [8]:
# Load and configure the VGG16 model
# vgg16 = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
# vgg16.trainable = False  # Freeze the layers

# Load and configure the ResNet50 model
resnet50 = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
resnet50.trainable = False  # Freeze the layers

# Combine features from both models
# combined = tf.keras.layers.concatenate([vgg16.output, resnet50.output])

# Add custom layers
flat = Flatten()(resnet50.output)
bn1 = BatchNormalization()(flat)
dense1 = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(bn1)
dropout = Dropout(0.5)(dense1)
bn2 = BatchNormalization()(dropout)
output = Dense(6, activation='softmax')(bn2)

# Create the final model
model = Model(inputs=resnet50.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=32, validation_split=0.2, epochs=10)

# Save the best model
model.save('vgg_resnet_model.h5')

Epoch 1/10
302/302 [==============================] - 468s 2s/step - loss: 3.1124 - accuracy: 0.2227 - val_loss: 2.3126 - val_accuracy: 0.2819
Epoch 2/10
302/302 [==============================] - 451s 1s/step - loss: 2.4673 - accuracy: 0.2843 - val_loss: 2.2800 - val_accuracy: 0.2964
Epoch 3/10
207/302 [===================>..........] - ETA: 1:54 - loss: 2.3002 - accuracy: 0.3261

KeyboardInterrupt: 

In [7]:
model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(6, activation='softmax')  # Assuming labels are one-hot encoded
])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model2.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32)

In [ ]:
# Evaluate the model
model2.evaluate([X_test, X_test], y_test)